<a href="https://colab.research.google.com/github/olgasem10/Unsafe/blob/main/Ray_with_default_hp_space.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
!pip install ray[tune]

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
import numpy as np
import pandas as pd

### Данные (с добавлением спец токена)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/internship/data/train_randst0.csv')
test_df = pd.read_csv('/content/drive/MyDrive/internship/data/val_randst0.csv')
#train_df = train_df[["text", "unsafe"]]
#test_df = test_df[["text", "unsafe"]]

In [ ]:
train_df = train_df.loc[(train_df['unsafe'] >= 0.8) | (train_df['unsafe'] <= 0.2)]

In [ ]:
translator = {
    'crime_real' : 'криминал',
    'crime_web' : 'веб-криминал',
    'drugs' : 'наркотики',
    'gambling' : 'азартные игры',
    'pornography' : 'порнография',
    'prostitution' : 'проституция',
    'slavery' : 'рабство',
    'suicide' : 'суицид',
    'terrorism' : 'терроризм',
    'weapons' : 'оружие', 
    'body_shaming' : 'уродства',
    'halth_shaming' : 'инвалидность',
    'politics' : 'политика',
    'racism' : 'расизм',
    'religion' : 'религия',
    'sex_minorities' : 'гомосексуализм',
    'sexism' : 'сексизм',
    'social' : 'социальное',
    'none' : ''
}

In [ ]:
col_names = ['crime_real',
 'crime_web',
 'drugs',
 'gambling',
 'pornography',
 'prostitution',
 'slavery',
 'suicide',
 'terrorism',
 'weapons',
 'body_shaming',
 'halth_shaming',
 'politics',
 'racism',
 'religion',
 'sex_minorities',
 'sexism',
 'social']

In [ ]:
def binary(val):
    return round(val)

In [ ]:
for name in col_names:
    train_df[name] = train_df[name].apply(binary)
    test_df[name] = test_df[name].apply(binary)

In [ ]:
train_df['unsafe'] = train_df['unsafe'].apply(binary)
test_df['unsafe'] = test_df['unsafe'].apply(binary)

In [ ]:
def add_special_tokens(dataframe):
    new_dataframe = dataframe.copy()
    for column in col_names:
        indices = dataframe.index[dataframe[column] == 1]
        new_dataframe.loc[indices, ['text']] = '<' + translator[column] + '> ' + dataframe.loc[indices, ['text']].astype(str)
    return new_dataframe

In [ ]:
train_df = add_special_tokens(train_df)
test_df = add_special_tokens(test_df)

In [ ]:
train_df = train_df.sample(frac=1)
test_df = test_df.sample(frac=1)

In [ ]:
x_train = train_df['text'].tolist()#[:15000]
y_train = train_df['unsafe'].tolist()#[:15000]
x_test = test_df['text'].tolist()#[:2000]
y_test = test_df['unsafe'].tolist()#[:2000]

In [ ]:
class UnsafeData(Dataset):

    def __init__(self, texts, targets, tokenizer, max_len):
        
        super().__init__()
        
        self.texts = texts
        self.targets = targets        
        self.max_len = max_len
        self.tokenizer = tokenizer

    def __len__(self):
        
        return len(self.texts)

    
    def __getitem__(self, index):
        x = self.texts[index]

        enc_dict = self.tokenizer(x, truncation=True, max_length=self.max_len, padding='max_length')
      
        item = {key: torch.tensor(val).long() for key, val in enc_dict.items()}
        item['labels'] = torch.tensor(self.targets[index]).long()

        return item 

### Обучение

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
model_name = 'DeepPavlov/rubert-base-cased-conversational'

In [ ]:
tokenizer = BertTokenizer.from_pretrained(model_name)

In [ ]:
train_dataset = UnsafeData(x_train, y_train, tokenizer, max_len = 60)
test_dataset = UnsafeData(x_test, y_test, tokenizer, max_len = 60)

In [ ]:
len(train_dataset), len(test_dataset)

(120120, 24501)

In [ ]:
train_dataset[10000]

{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'input_ids': tensor([  101,   160, 21749,   130, 48931,   164, 40051,   477,   507,   465,
         58496,   439, 28019, 14646, 32260,  8297, 41387,   128,  1007,  1151,
         19644,   340,   362,  5703,   166,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]),
 'labels': tensor(1),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}

In [ ]:
def model_init():
    return BertForSequenceClassification.from_pretrained(model_name, return_dict=True)

In [ ]:
training_args = TrainingArguments(
    output_dir='/bert1',
    logging_steps = 600,
    evaluation_strategy = 'steps',
    logging_dir='/bert1/logs',
    do_eval=True,
    save_steps = 10000,
    save_total_limit = 2,
    disable_tqdm=True
)

trainer = Trainer(
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    model_init=model_init,
    compute_metrics=compute_metrics,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-conversational and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
### По какой метрике оптимизировать

def my_func(metrics):
    return metrics['eval_f1']

In [ ]:
import ray

In [ ]:
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.schedulers import AsyncHyperBandScheduler

In [ ]:
best_run = trainer.hyperparameter_search(
    n_trials=20,
    compute_objective = my_func,
    direction="maximize",
    backend = 'ray',
    search_alg=HyperOptSearch(metric = 'eval_f1', mode = 'max'),
    scheduler=AsyncHyperBandScheduler(metric = 'eval_f1', mode = 'max')
    )

2021-03-18 08:38:57,950	INFO services.py:1174 -- View the Ray dashboard at http://127.0.0.1:8265
2021-03-18 08:39:03,399	WARNING function_runner.py:541 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


== Status ==
Memory usage on this node: 3.8/12.7 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 64.000: None | Iter 16.000: None | Iter 4.000: None | Iter 1.000: None
Resources requested: 1/2 CPUs, 1/1 GPUs, 0.0/7.32 GiB heap, 0.0/2.49 GiB objects (0/1.0 accelerator_type:T4)
Result logdir: /root/ray_results/_inner_2021-03-18_08-39-03
Number of trials: 1/20 (1 RUNNING)
+-----------------+----------+-------+-----------------+--------------------+-------------------------------+--------+
| Trial name      | status   | loc   |   learning_rate |   num_train_epochs |   per_device_train_batch_size |   seed |
|-----------------+----------+-------+-----------------+--------------------+-------------------------------+--------|
| _inner_64e82530 | RUNNING  |       |     9.69583e-05 |                  1 |                            64 | 34.923 |
+-----------------+----------+-------+-----------------+--------------------+-------------------------------+--------+




(pid=366) 2021-03-18 08:39:04.677449: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
(pid=366) Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-conversational and are newly initialized: ['classifier.bias', 'classifier.weight']
(pid=366) You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


(pid=366) {'loss': 0.416, 'learning_rate': 6.596470393691822e-05, 'epoch': 0.32}
Result for _inner_64e82530:
  date: 2021-03-18_08-48-56
  done: false
  epoch: 0.32
  eval_accuracy: 0.7911513815762622
  eval_f1: 0.785431839440515
  eval_loss: 0.4543006420135498
  eval_precision: 0.7839843424239313
  eval_recall: 0.7911513815762622
  eval_runtime: 106.523
  eval_samples_per_second: 230.007
  experiment_id: fc3e59f0abfc4481a821dc7f9db32f3a
  hostname: b5916f891e08
  iterations_since_restore: 1
  node_ip: 172.28.0.2
  objective: 0.785431839440515
  pid: 366
  time_since_restore: 592.135365486145
  time_this_iter_s: 592.135365486145
  time_total_s: 592.135365486145
  timestamp: 1616057336
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 64e82530
  
(pid=366) {'eval_loss': 0.4543006420135498, 'eval_accuracy': 0.7911513815762622, 'eval_f1': 0.785431839440515, 'eval_precision': 0.7839843424239313, 'eval_recall': 0.7911513815762622, 'eval_runtime': 106.523, 'eval_samples_per_se

(pid=365) 2021-03-18 09:08:35.052139: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
(pid=365) Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-conversational and are newly initialized: ['classifier.weight', 'classifier.bias']
(pid=365) You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

(pid=1760) {'loss': 0.5924, 'learning_rate': 3.533178645919406e-05, 'epoch': 0.04}
Result for _inner_a2598198:
  date: 2021-03-18_13-34-14
  done: true
  epoch: 0.04
  eval_accuracy: 0.7004612056650749
  eval_f1: 0.5773008286881254
  eval_loss: 0.619695782661438
  eval_precision: 0.6404496922483778
  eval_recall: 0.7004612056650749
  eval_runtime: 104.149
  eval_samples_per_second: 235.25
  experiment_id: e33d229d3bfc40579b9d300208dfdad5
  hostname: b5916f891e08
  iterations_since_restore: 1
  node_ip: 172.28.0.2
  objective: 0.5773008286881254
  pid: 1760
  time_since_restore: 211.84330439567566
  time_this_iter_s: 211.84330439567566
  time_total_s: 211.84330439567566
  timestamp: 1616074454
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: a2598198
  
== Status ==
Memory usage on this node: 5.6/12.7 GiB
Using AsyncHyperBand: num_stopped=17
Bracket: Iter 64.000: None | Iter 16.000: None | Iter 4.000: 0.8039032398790328 | Iter 1.000: 0.7773021866462699
Resources requeste

(pid=1789) 2021-03-18 13:34:16.219802: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
(pid=1789) Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-conversational and are newly initialized: ['classifier.bias', 'classifier.weight']
(pid=1789) You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


(pid=1789) {'loss': 0.611, 'learning_rate': 6.195295774667361e-05, 'epoch': 0.04}


(pid=1789) /usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
(pid=1789)   _warn_prf(average, modifier, msg_start, len(result))
2021-03-18 13:37:47,937	INFO tune.py:450 -- Total run time: 17930.90 seconds (17924.42 seconds for the tuning loop).


Result for _inner_22a5f49e:
  date: 2021-03-18_13-37-47
  done: true
  epoch: 0.04
  eval_accuracy: 0.7004612056650749
  eval_f1: 0.5770739126622669
  eval_loss: 0.6193544268608093
  eval_precision: 0.4906459006417703
  eval_recall: 0.7004612056650749
  eval_runtime: 104.8177
  eval_samples_per_second: 233.749
  experiment_id: c3721b845dd04f21b730852c768a13ba
  hostname: b5916f891e08
  iterations_since_restore: 1
  node_ip: 172.28.0.2
  objective: 0.5770739126622669
  pid: 1789
  time_since_restore: 212.35546588897705
  time_this_iter_s: 212.35546588897705
  time_total_s: 212.35546588897705
  timestamp: 1616074667
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 22a5f49e
  
(pid=1789) {'eval_loss': 0.6193544268608093, 'eval_accuracy': 0.7004612056650749, 'eval_f1': 0.5770739126622669, 'eval_precision': 0.4906459006417703, 'eval_recall': 0.7004612056650749, 'eval_runtime': 104.8177, 'eval_samples_per_second': 233.749, 'epoch': 0.04}
== Status ==
Memory usage on this node

#### Пример запуска на части датасета

In [ ]:
best_run = trainer.hyperparameter_search(
    n_trials=10,
    compute_objective = my_func,
    direction="maximize",
    backend = 'ray',
    search_alg=HyperOptSearch(metric = 'eval_f1', mode = 'max'),
    scheduler=AsyncHyperBandScheduler(metric = 'eval_f1', mode = 'max')
    )

== Status ==
Memory usage on this node: 4.3/12.7 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 64.000: None | Iter 16.000: None | Iter 4.000: None | Iter 1.000: None
Resources requested: 1/2 CPUs, 1/1 GPUs, 0.0/7.42 GiB heap, 0.0/2.54 GiB objects (0/1.0 accelerator_type:T4)
Result logdir: /root/ray_results/_inner_2021-03-17_11-51-38
Number of trials: 1/10 (1 RUNNING)
+-----------------+----------+-------+-----------------+--------------------+-------------------------------+---------+
| Trial name      | status   | loc   |   learning_rate |   num_train_epochs |   per_device_train_batch_size |    seed |
|-----------------+----------+-------+-----------------+--------------------+-------------------------------+---------|
| _inner_2182b94e | RUNNING  |       |     9.39003e-05 |                  1 |                            32 | 36.0425 |
+-----------------+----------+-------+-----------------+--------------------+-------------------------------+---------+




(pid=947) 2021-03-17 11:51:39.072799: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
(pid=947) Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-conversational and are newly initialized: ['classifier.weight', 'classifier.bias']
(pid=947) You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


(pid=947) {'train_runtime': 183.8865, 'train_samples_per_second': 2.55, 'epoch': 1.0}
(pid=947) {'eval_loss': 0.4715965688228607, 'eval_accuracy': 0.789, 'eval_f1': 0.7880768352084142, 'eval_precision': 0.7872916009785267, 'eval_recall': 0.789, 'eval_runtime': 7.9616, 'eval_samples_per_second': 251.206, 'epoch': 1.0}
Result for _inner_2182b94e:
  date: 2021-03-17_11-55-03
  done: true
  epoch: 1.0
  eval_accuracy: 0.789
  eval_f1: 0.7880768352084142
  eval_loss: 0.4715965688228607
  eval_mem_cpu_alloc_delta: 137517
  eval_mem_cpu_peaked_delta: 231704
  eval_mem_gpu_alloc_delta: 0
  eval_mem_gpu_peaked_delta: 16966144
  eval_precision: 0.7872916009785267
  eval_recall: 0.789
  eval_runtime: 7.9616
  eval_samples_per_second: 251.206
  experiment_id: 69e5b319493a458d9adb9246d52b9882
  hostname: 30101e747662
  iterations_since_restore: 1
  node_ip: 172.28.0.2
  objective: 0.7880768352084142
  pid: 947
  time_since_restore: 204.9747622013092
  time_this_iter_s: 204.9747622013092
  time_tota

(pid=946) 2021-03-17 11:55:04.712475: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
(pid=946) Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-conversational and are newly initialized: ['classifier.weight', 'classifier.bias']
(pid=946) You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


(pid=946) {'loss': 0.5517, 'learning_rate': 1.097425249072027e-06, 'epoch': 1.28}
Result for _inner_21872f74:
  date: 2021-03-17_11-59-22
  done: true
  epoch: 1.28
  eval_accuracy: 0.735
  eval_f1: 0.7408069877883311
  eval_loss: 0.5122585296630859
  eval_precision: 0.7513574845827476
  eval_recall: 0.735
  eval_runtime: 7.9894
  eval_samples_per_second: 250.33
  experiment_id: e3691bd74f54474aaed34c8e2dd52f43
  hostname: 30101e747662
  iterations_since_restore: 1
  node_ip: 172.28.0.2
  objective: 0.7408069877883311
  pid: 946
  time_since_restore: 258.65727972984314
  time_this_iter_s: 258.65727972984314
  time_total_s: 258.65727972984314
  timestamp: 1615982362
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 21872f74
  
== Status ==
Memory usage on this node: 6.1/12.7 GiB
Using AsyncHyperBand: num_stopped=1
Bracket: Iter 64.000: None | Iter 16.000: None | Iter 4.000: None | Iter 1.000: 0.7762593733533935
Resources requested: 0/2 CPUs, 0/1 GPUs, 0.0/7.42 GiB heap, 0

...

(pid=1419) {'loss': 0.4944, 'learning_rate': 2.1535492040104403e-05, 'epoch': 0.32}
Result for _inner_1bf85a56:
  date: 2021-03-17_12-31-53
  done: true
  epoch: 0.32
  eval_accuracy: 0.742
  eval_f1: 0.7503533246414602
  eval_loss: 0.4928911626338959
  eval_precision: 0.7729951551386455
  eval_recall: 0.742
  eval_runtime: 7.9808
  eval_samples_per_second: 250.6
  experiment_id: 71126dabd6c9496a883dbf8f18902818
  hostname: 30101e747662
  iterations_since_restore: 1
  node_ip: 172.28.0.2
  objective: 0.7503533246414602
  pid: 1419
  time_since_restore: 112.51147770881653
  time_this_iter_s: 112.51147770881653
  time_total_s: 112.51147770881653
  timestamp: 1615984313
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 1bf85a56
  
== Status ==
Memory usage on this node: 6.1/12.7 GiB
Using AsyncHyperBand: num_stopped=7
Bracket: Iter 64.000: None | Iter 16.000: None | Iter 4.000: None | Iter 1.000: 0.7748205457248436
Resources requested: 0/2 CPUs, 0/1 GPUs, 0.0/7.42 GiB heap,

(pid=1452) 2021-03-17 12:31:55.351798: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
(pid=1452) Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-conversational and are newly initialized: ['classifier.bias', 'classifier.weight']
(pid=1452) You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


(pid=1452) {'loss': 0.5537, 'learning_rate': 1.651245813229014e-05, 'epoch': 0.16}


2021-03-17 12:33:25,783	INFO tune.py:450 -- Total run time: 2507.68 seconds (2507.66 seconds for the tuning loop).


Result for _inner_7d8d3bb0:
  date: 2021-03-17_12-33-25
  done: true
  epoch: 0.16
  eval_accuracy: 0.757
  eval_f1: 0.7366643352449901
  eval_loss: 0.6576597094535828
  eval_precision: 0.7429200033513469
  eval_recall: 0.757
  eval_runtime: 7.9077
  eval_samples_per_second: 252.917
  experiment_id: bd494ddec0ed4499a77e781a75dba360
  hostname: 30101e747662
  iterations_since_restore: 1
  node_ip: 172.28.0.2
  objective: 0.7366643352449901
  pid: 1452
  time_since_restore: 91.10976338386536
  time_this_iter_s: 91.10976338386536
  time_total_s: 91.10976338386536
  timestamp: 1615984405
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 7d8d3bb0
  
(pid=1452) {'eval_loss': 0.6576597094535828, 'eval_accuracy': 0.757, 'eval_f1': 0.7366643352449901, 'eval_precision': 0.7429200033513469, 'eval_recall': 0.757, 'eval_runtime': 7.9077, 'eval_samples_per_second': 252.917, 'epoch': 0.16}
== Status ==
Memory usage on this node: 6.1/12.7 GiB
Using AsyncHyperBand: num_stopped=8
Bracket:

In [ ]:
best_run

BestRun(run_id='2182b94e', objective=0.7880768352084142, hyperparameters={'learning_rate': 9.390028242887286e-05, 'num_train_epochs': 1, 'seed': 36.04249333663025, 'per_device_train_batch_size': 32})

In [ ]:
#!rm -r ./bert1